# Summary of best models for given numbers of layers

In [ ]:
# Each row is the output for a model in the following format. Different lines for different model structures. See the cell below for the excecuted codes.
# print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}, {criteria:.4f}],')
# Example: [16layers, 12(6)heads, hidden-dim=696, inter-layer-dim = 696x4, head-dim=224, #param=218million, val_loss_prev, #epochs, val_loss_current, val_loss_prev_prev, train_loss, Num-steps]

[+18, 6, 3, 720, 2880, 224, 200.6,+2.4402, 20, 2.4407, 2.4406, 2.2662,140000, 0.0005], # minGemma-hidden_layers18-att_heads6-kv_heads3-hidden720-intermediate2880-head_dim224-T512--2025-08-10-05-50.pth
# L16
[+12,10, 5, 720, 2880, 256, 177.3,+2.4416, 29, 2.4417, 2.4418, 2.2870, 70000, 0.0005], # minGemma-hidden_layers12-att_heads10-kv_heads5-hidden720-intermediate2880-head_dim256-T512--2025-07-14-22-22.pth
[ 10, 8, 4, 736, 3680, 256, 163.6, 2.4597, 11, 2.4594, 2.4605, 2.3522, 70000], # w/o last drop ???? not best???
[  8, 8, 4, 800, 4800, 480, 206.2,+2.4628, 22,   stop, 2.4631, 2.3529, 80000], # w/o last drop

# L36 (Models with 36 Layers)

# L30

# L24

# L18

In [ ]:
# L18 Forgetter with last drop (sleep2 in the end) (default: x4 B28 lr8e-4 WD0.25 N70000)

[ 18, 8, 4, 640, 2560, 256, 191.5, 2.4406, 19, 2.4407, 2.4420, 2.2828, 140000, 0.0005], # 15.7h 15.0G F1:2.4420@18th F2@19th:-0.0014 # Normal100M # Grad-Acc and B2 fixed (use this program from now on)
# F1-best: minGemma-hidden_layers18-att_heads8-kv_heads4-hidden640-intermediate2560-head_dim256-T512--2025-07-27-11-16.pth
# F2-best: minGemma-hidden_layers18-att_heads8-kv_heads4-hidden640-intermediate2560-head_dim256-T512--2025-07-28-03-16.pth

[+18, 6, 3, 720, 2880, 224, 200.6,+2.4402, 20, 2.4407, 2.4406, 2.2662, 140000, 0.0005], # 14.7h 15.3G F1:2.4440@13th F2@14th:-0.0010  # [3.0504Normal10M]
# F1-best: minGemma-hidden_layers18-att_heads6-kv_heads3-hidden720-intermediate2880-head_dim224-T512--2025-08-05-21-18.pth
# F2-best: minGemma-hidden_layers18-att_heads6-kv_heads3-hidden720-intermediate2880-head_dim224-T512--2025-08-10-05-50.pth

In [2]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  18 # 8 # 8 # G28 G18 #blocks
num_attention_heads = 6 # 4 # G16 G8
num_key_value_heads = 3 # 4 # G16 G1
hidden_size = num_attention_heads*120 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 224 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r'); torch.cuda.empty_cache()
T=512; B=28//2; N_step=70000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# F2(F1) criteria style
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100; flag=0; criteria=0.0005 # * 4 * 4 * 4 # count = all, 5, 3~4, 2
for ep in range(Max_ep):
    if flag==1: # Forgetter_2
        print("F2"); del model; torch.cuda.empty_cache(); model = minGemma().to(device); model.load_state_dict(torch.load('test.pth'))
    training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=8e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"]
    loss = []; model.eval(); B2=28; B2=12; torch.cuda.empty_cache() # Evaluation follows
    for k in range(6000): # std=0.0056 for 1000 with 89sec #5min
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); print(ep+1, tloss, loss_current); del common; torch.cuda.empty_cache(); model.train()
    if loss_current < 2.4444:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= loss_prev) and (flag==1):
        break
    if (loss_current >= (loss_prev-criteria)) and (flag==0):
        flag=1
        if loss_current >= loss_prev:
            model.load_state_dict(torch.load('test.pth')) # back to previous
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss; torch.save(model.state_dict(), 'test.pth'); # always save
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}, {criteria:.4f}],')

1003.52
L18 att6 kv_heads3 hidden720 intermediate2880 head_dim224 T512


Step,Training Loss
70000,2.780800


1 2.7807924107142856 tensor(2.5139)


Step,Training Loss
70000,2.344600


2 2.3446390625 tensor(2.4765)


...

19 2.2659946428571427 tensor(2.4406)
F2


Step,Training Loss
70000,2.266200


20 2.2661977678571428 tensor(2.4402)
F2


Step,Training Loss
70000,2.265800


21 2.2658053571428574 tensor(2.4407)
[ 18, 6, 3, 720, 2880, 224, 200.6, 2.4402, 20, 2.4407, 2.4406, 2.2662, 140000, 0.0005],


In [1]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  18 # 8 # 8 # G28 G18 #blocks
num_attention_heads = 8 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*80 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 256 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r'); torch.cuda.empty_cache()
T=512; B=28//2; N_step=70000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# F2(F1) criteria style
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100; flag=0; criteria=0.0005 # * 4 * 4 * 4 # count = all, 5, 3~4, 2
for ep in range(Max_ep):
    if flag==1: # Forgetter_2
        print("F2"); del model; torch.cuda.empty_cache(); model = minGemma().to(device); model.load_state_dict(torch.load('test.pth'))
    training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=8e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"]
    loss = []; model.eval(); B2=28; B2=12; torch.cuda.empty_cache() # Evaluation follows
    for k in range(6000): # std=0.0056 for 1000 with 89sec #5min
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); print(ep+1, tloss, loss_current); del common; torch.cuda.empty_cache(); model.train()
    if loss_current < 2.4444:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= loss_prev) and (flag==1):
        break
    if (loss_current >= (loss_prev-criteria)) and (flag==0):
        flag=1
        if loss_current >= loss_prev:
            model.load_state_dict(torch.load('test.pth')) # back to previous
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss; torch.save(model.state_dict(), 'test.pth'); # always save
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}, {criteria:.4f}],')

C:\Users\miura\anaconda3\envs\minGemma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1003.52
L18 att8 kv_heads4 hidden640 intermediate2560 head_dim256 T512


C:\Users\miura\AppData\Local\Temp\ipykernel_17092\733028177.py:42: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs


Step,Training Loss
70000,2.742300


1 2.7422828125 tensor(2.5205)


...

18 2.2797073660714284 tensor(2.4420)
F2


Step,Training Loss
70000,2.282800


19 2.2828464285714287 tensor(2.4406)
F2


Step,Training Loss
70000,2.281100


20 2.281074330357143 tensor(2.4407)
[ 18, 8, 4, 640, 2560, 256, 191.5, 2.4406, 19, 2.4407, 2.4420, 2.2828, 140000, 0.0005],


# L16

# L12

In [ ]:
# L12 Forgetter with last drop (sleep2 in the end) (default: x4 B28 lr8e-4 WD0.25 N70000)

[ 12, 12, 4, 768, 3072, 192, 180.3, 2.4432, 18, 2.4433, 2.4433, 2.2734, 70000, 0.0005], # 12.7h 22.5G F1:2.4444@15th F2@16th:-0.0009 
# F1: minGemma-hidden_layers12-att_heads12-kv_heads4-hidden768-intermediate3072-head_dim192-T512--2025-06-26-12-17.pth
# F2: minGemma-hidden_layers12-att_heads12-kv_heads4-hidden768-intermediate3072-head_dim192-T512--2025-06-28-02-52.pth

[+12, 10, 5, 720, 2880, 256, 177.3,+2.4416, 29, 2.4417, 2.4418, 2.2870, 70000, 0.0005], # 13.2h 22.4G F1:2.4446@17th F2@18th:-0.0003
# F1: minGemma-hidden_layers12-att_heads10-kv_heads5-hidden720-intermediate2880-head_dim256-T512--2025-07-08-04-22.pth
# F2: minGemma-hidden_layers12-att_heads10-kv_heads5-hidden720-intermediate2880-head_dim256-T512--2025-07-14-22-22.pth

In [2]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  12 # 8 # 8 # G28 G18 #blocks
num_attention_heads =10 # 4 # G16 G8
num_key_value_heads = 5 # 4 # G16 G1
hidden_size = num_attention_heads*72 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 256 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r'); torch.cuda.empty_cache()
T=512; B=28; N_step=70000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# F2(F1) criteria style
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100; flag=0; criteria=0.0005 # * 4 * 4 * 4 # count = all, 5, 3~4, 2
for ep in range(Max_ep):
    if flag==1: # Forgetter_2
        print("F2"); del model; torch.cuda.empty_cache(); model = minGemma().to(device); model.load_state_dict(torch.load('test.pth'))
    training_args = TrainingArguments(learning_rate=8e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"]
    loss = []; model.eval(); B2=28; torch.cuda.empty_cache() # Evaluation follows
    for k in range(2000): # 6000 # std=0.0056 for 1000 with 89sec #5min
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); print(ep+1, tloss, loss_current); del common; torch.cuda.empty_cache(); model.train()
    if loss_current < 2.4688:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= loss_prev) and (flag==1):
        break
    if (loss_current >= (loss_prev-criteria)) and (flag==0):
        flag=1
        if loss_current >= loss_prev:
            model.load_state_dict(torch.load('test.pth')) # back to previous
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss; torch.save(model.state_dict(), 'test.pth'); # always save
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}, {criteria:.4f}],')

1003.52
L12 att10 kv_heads5 hidden720 intermediate2880 head_dim256 T512


Step,Training Loss
70000,2.773800


1 2.773804017857143 tensor(2.5152)


Step,Training Loss
70000,2.353900


2 2.3538703125 tensor(2.4759)


...

28 2.2867613839285714 tensor(2.4418)
F2


Step,Training Loss
70000,2.287000


29 2.286983482142857 tensor(2.4416)
F2


Step,Training Loss
70000,2.286900


30 2.286904464285714 tensor(2.4417)
[ 12, 10, 5, 720, 2880, 256, 177.3, 2.4416, 29, 2.4417, 2.4418, 2.2870, 70000, 0.0005],


In [1]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  12 # 8 # 8 # G28 G18 #blocks
num_attention_heads =12 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*768//12 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 192 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r'); torch.cuda.empty_cache()
T=512; B=28; N_step=70000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# F2(F1) criteria style
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100; flag=0; criteria=0.0005 # * 4 * 4 * 4 # count = all, 5, 3~4, 2
for ep in range(Max_ep):
    if flag==1: # Forgetter_2
        print("F2"); del model; torch.cuda.empty_cache(); model = minGemma().to(device); model.load_state_dict(torch.load('test.pth'))
    training_args = TrainingArguments(learning_rate=8e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"]
    loss = []; model.eval(); B2=28; torch.cuda.empty_cache() # Evaluation follows
    for k in range(2000): # 6000 # std=0.0056 for 1000 with 89sec #5min
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); print(ep+1, tloss, loss_current); del common; torch.cuda.empty_cache(); model.train()
    if loss_current < 2.4688:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= loss_prev) and (flag==1):
        break
    if (loss_current >= (loss_prev-criteria)) and (flag==0):
        flag=1
        if loss_current >= loss_prev:
            model.load_state_dict(torch.load('test.pth')) # back to previous
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss; torch.save(model.state_dict(), 'test.pth'); # always save
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}, {criteria:.4f}],')

C:\Users\miura\anaconda3\envs\minGemma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1003.52
L12 att12 kv_heads4 hidden768 intermediate3072 head_dim192 T512


C:\Users\miura\AppData\Local\Temp\ipykernel_4880\3747344726.py:42: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs


Step,Training Loss
70000,2.781200


1 2.7812066964285713 tensor(2.5154)


...

17 2.27433125 tensor(2.4433)
F2


Step,Training Loss
70000,2.273400


18 2.2733881696428573 tensor(2.4432)
F2


Step,Training Loss
70000,2.273000


19 2.2730154017857145 tensor(2.4433)
[ 12, 12, 4, 768, 3072, 192, 180.3, 2.4432, 18, 2.4433, 2.4433, 2.2734, 70000, 0.0005],


In [ ]:
# (LEGACY) # L12 Forgetter without last drop (no sleep2 in the end) (default: B28 lr10e-4 WD0.25)
[+12,12, 4, 792, 3168, 224, 198.4, 2.4490, 12,+2.4485, 2.4498, 2.3312, 70000], # 14.5h 23.6G x4 TOO big
[ 12, 8, 4, 736, 3680, 256, 188.9, 2.4521, 11, 2.4516, 2.4529, 2.3373, 70000], # 12.4h 23.2G x5

In [1]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  12 # 8 # 8 # G28 G18 #blocks
num_attention_heads =12 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*792//12 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 224 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=70000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# Forgetter with Hand-eval
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100
for ep in range(Max_ep):
    training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"] # trainer = Trainer(model=model, args=training_args, eval_dataset=torch.utils.data.TensorDataset(torch.randint(len(val_data)-T-1, (12*400*4,))), data_collator=map_to_array_Val); trainer.can_return_loss = True; loss_current = trainer.evaluate()["eval_loss"]; print(ep+1, tloss, loss_current)
    loss = []; model.eval(); B2=16;
    for k in range(4000): #1000LP # std=0.0056 for 1000 with 89sec (for B12? and total7200)
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); model.train(); del common; print(loss_current)
    if loss_current < 2.4542:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= (loss_prev - 0.0005)): #(loss_current >= loss_prev_prev) and (loss_prev >= loss_prev_prev):
        break
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}],')

C:\Users\miura\anaconda3\envs\minGemma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1003.52
L12 att12 kv_heads4 hidden792 intermediate3168 head_dim224 T512


C:\Users\miura\AppData\Local\Temp\ipykernel_7940\2203527428.py:42: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs


Step,Training Loss
70000,2.803100


tensor(2.5088)


...

tensor(2.4498)


Step,Training Loss
70000,2.331200


tensor(2.4490)


Step,Training Loss
70000,2.330900


tensor(2.4485)
[ 12, 12, 4, 792, 3168, 224, 198.4, 2.4490, 12, 2.4485, 2.4498, 2.3312, 70000],


# L10 (Not used for figure because not explored enough)

In [ ]:
# (LEGACY) # L8 Forgetter without last drop (no sleep2 in the end) (default: B28 lr10e-4 WD0.25)
[ 10, 8, 4, 736, 3680, 256, 163.6, 2.4597, 11, 2.4594, 2.4605, 2.3522, 70000], # 10.6h 21.4G x5

# L8 (Not used for figure because not explored enough)

In [ ]:
# (LEGACY) # L8 Forgetter without last drop (no sleep2 in the end) (default: B28 lr10e-4 WD0.25)
[ 8, 8, 4, 800, 4800, 480, 206.2, 2.4653, 14, 2.4648, 2.4654, 2.3556, 80000], # 16.0h 23.4G x6 # Ended when decrease is smaller than 0.0005  
[ 8, 8, 4, 800, 4800, 480, 206.2,+2.4628, 22,   stop, 2.4631, 2.3529, 80000], # Same run as above but looked at longer than 0.0005 criteria
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.4732, 10, 2.4729, 2.4743, 2.3756, 70000], # 9.0h 19.4G x5

In [1]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =   8 # 8 # G28 G18 #blocks
num_attention_heads = 8 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*100 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*6 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 480 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=32; B=28; N_step=80000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# Forgetter with Hand-eval
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100
for ep in range(Max_ep): # * (Max_ep-ep/99999)/Max_ep
    training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"] # trainer = Trainer(model=model, args=training_args, eval_dataset=torch.utils.data.TensorDataset(torch.randint(len(val_data)-T-1, (12*400*4,))), data_collator=map_to_array_Val); trainer.can_return_loss = True; loss_current = trainer.evaluate()["eval_loss"]; print(ep+1, tloss, loss_current)
    loss = []; model.eval(); B2=16;
    for k in range(4000): #1000LP # std=0.0056 for 1000 with 89sec (for B12? and total7200)
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); model.train(); del common; print(loss_current)
    if loss_current < 2.4542:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= loss_prev): #(loss_current >= loss_prev_prev) and (loss_prev >= loss_prev_prev):
        break
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}],')

C:\Users\miura\anaconda3\envs\minGemma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1146.88
L8 att8 kv_heads4 hidden800 intermediate4800 head_dim480 T512


C:\Users\miura\AppData\Local\Temp\ipykernel_17944\4083251127.py:42: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs


Step,Training Loss
80000,3.058200


tensor(2.5848)


...

tensor(2.4631)


Step,Training Loss
80000,2.352900


tensor(2.4628)


Step,Training Loss


KeyboardInterrupt: 

In [3]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =   8 # 8 # G28 G18 #blocks
num_attention_heads = 8 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*92 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*5 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 256 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=70000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')

# Forgetter with Hand-eval
loss_prev=10000.0; loss_prev_prev=loss_prev; tloss_prev=10000.0; Max_ep=100
for ep in range(Max_ep):
    training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
    trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
    result = trainer.train(); tloss=result[2]["train_loss"] # trainer = Trainer(model=model, args=training_args, eval_dataset=torch.utils.data.TensorDataset(torch.randint(len(val_data)-T-1, (12*400*4,))), data_collator=map_to_array_Val); trainer.can_return_loss = True; loss_current = trainer.evaluate()["eval_loss"]; print(ep+1, tloss, loss_current)
    loss = []; model.eval(); B2=16;
    for k in range(4000): #1000LP # std=0.0056 for 1000 with 89sec (for B12? and total7200)
        val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
        loss += [model(common.to('cuda', non_blocking=True))[0].item()]
    loss_current = torch.Tensor(loss).mean(); model.train(); del common; print(loss_current)
    if loss_current < 2.4542:
        torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
    if (loss_current >= (loss_prev - 0.0005)): #(loss_current >= loss_prev_prev) and (loss_prev >= loss_prev_prev):
        break
    loss_prev_prev = loss_prev; loss_prev = loss_current; tloss_prev=tloss
print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {loss_prev:.4f}, {ep}, {loss_current:.4f}, {loss_prev_prev:.4f}, {tloss_prev:.4f}, {N_step}],')

1003.52
L8 att8 kv_heads4 hidden736 intermediate3680 head_dim256 T512


Step,Training Loss
70000,2.834300


tensor(2.5342)


Step,Training Loss
70000,2.426200


tensor(2.5000)


...

tensor(2.4743)


Step,Training Loss
70000,2.375600


tensor(2.4732)


Step,Training Loss
70000,2.374800


tensor(2.4729)
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.4732, 10, 2.4729, 2.4743, 2.3756, 70000],


# L6 (Not used for figure because not explored enough)

In [ ]:
# L6 Forgetter without last drop (no sleep2 in the end) (default: B28? WD1x lr10e-4x)


# Baby10M
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0624,  6, 3.0633, 3.0648, 2.7113, 20000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0434,  9, 3.0437, 3.0438, 2.7142, 15000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0413, 13, 3.0423, 3.0424, 2.7203, 12000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0435, 10, 3.0438, 3.0446, 2.7185, 11000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0410, 12, 3.0413, 3.0420, 2.7203, 10500],
[+6, 8, 4, 768, 3072, 256, 109.4,+3.0407, 12, 3.0411, 3.0409, 2.7164, 10000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0440, 13, 3.0440, 3.0446, 2.7216,  9500],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0484, 10, 3.0490, 3.0499, 2.7331,  9000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0418, 11, 3.0419, 3.0425, 2.7351,  8000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0489, 19, 3.0490, 3.0492, 2.7279,  7000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0563, 13, 3.0568, 3.0580, 2.7389,  6000],
[ 6, 8, 4, 768, 3072, 256, 109.4, 3.0681, 11, 3.0682, 3.0703, 2.7538,  5000],